In [ ]:
include("../load.jl")
include("../../data/baron/minlp_demo.jl")
include("../tools/plots.jl")
using Plots
using LaTeXStrings

In [ ]:
# BARON solution 
gm = minlp_demo(false);
set_optimizer(gm, BARON_SILENT)
optimize!(gm);
solution(gm)

In [ ]:
gm = minlp_demo(true)
set_optimizer(gm, CPLEX_SILENT)
set_param(gm, :ignore_accuracy, true)
set_param(gm, :step_size, 1e-4)
set_param(gm, :abstol, 1e-2)
x = gm.model[:x]
# Actually trying to optimize...
# find_bounds!(gm, all_bounds=true)
uniform_sample_and_eval!(gm)
learn_constraint!(gm.bbls, max_depth = 3)
globalsolve!(gm)

In [ ]:
# Showing the learners
idx = 2
bbr = gm.bbls[idx]
bbr.learners[end]

In [ ]:
# Feasible and infeasible points of constraint 1
bbc = gm.bbls[1]
feas_idxs = findall(x -> x .>= 0, bbc.Y)
infeas_idxs = findall(x -> x .< 0, bbc.Y)
scatter(bbc.X[feas_idxs,1],
bbc.X[feas_idxs, 2], bbc.X[feas_idxs, 3], 
        legend = false, label = "feasible", color = :green,  fmt = :png)
plt = scatter!(bbc.X[infeas_idxs,1],
bbc.X[infeas_idxs, 2], bbc.X[infeas_idxs, 3], 
        title = "Constraint samples",
xlabel = L"x_1", ylabel = L"x_2", zlabel = L"x_3", 
legend = true, label = "infeasible", color = :red,  fmt = :png)
# savefig(plt, "constr")

In [ ]:
# Showing the projected gradient descent iterations
# on the surface of constraint 1. 
symbs = [Symbol("x[$(i)]") for i=1:3]
npts = 100
f(x,y) = (0.8 * log(y + 1) + 0.96 * log((x - y) + 1))/0.8
plt = plot(range(0.35, stop = 1, length = npts),
      range(0.35, stop = 1, length = npts),
    zlims = [0.3,0.75],
    f, st=:surface, legend = false)
scatter!(gm.solution_history[1:40,symbs[1]],
    gm.solution_history[1:40,symbs[2]], 
    gm.solution_history[1:40,symbs[3]], xlabel = L"x_1", ylabel = L"x_2", zlabel = L"x_3", 
    xticks = 0:0.25:1, yticks = 0:0.25:1,
    title = "Projected gradient descent iterations",
    fmt = png, color = colormap("Blues", 40), legend = false)

In [ ]:
# Solution history, for good measure. 
println("First solution:" * string(gm.solution_history[80, :]))
println("")
println("First cost: " * string(gm.cost[1]))
println("Final solution:" * string(gm.solution_history[end, :]))
println("Final cost: " * string(gm.cost[end]))